# SmoothQuant on OPT-13B

### Guangxuan Xiao\*, Ji Lin\*, Mickael Seznec, Julien Demouth, Song Han

In this notebook, we use OPT-13B model to demonstrate SmoothQuant can use 8-bit for both weights and activations to achieve the same accuracy as FP16 models. Unlike previous method [[Dettmers *et al.*, 2022]](https://arxiv.org/abs/2208.07339), SmoothQuant enables fully INT8 GEMMs for linear layers and does not require high precision numbers to represent outliers. 

This notebook demonstrates SmoothQuant on OPT-13B in consideration of the user's resouce constraints. We have tested SmoothQuant on up to 176 billion parameter models (OPT-175B, BLOOM-176B, GLM-130B). You can also adjust the model name to validate SmoothQuant on other models. `../act_scales/` provides the activation channel scales for OPT and BLOOM models.

In order to run this notebook, you need to install the following packages:

- smoothquant
- PyTorch
- Transformers
- Accelerate

In [10]:
import importlib
from nanoquant.investigate import sweep, report_sweep
# Force reimport of the module
importlib.reload(importlib.import_module("nanoquant.investigate"))
importlib.reload(importlib.import_module("smoothquant.fake_quant"))

repo_dir = "smoothquant"
short_model_name = "opt-125m"
sweep(short_model_name=short_model_name, repo_dir=repo_dir, save_dir=".")
report_sweep(short_model_name=short_model_name, save_dir=".")


# n_bits = 8
# q_group_size = 0 # 0 means no grouping
# q_protect = False # False means no protection
# q_protection_scale = 0.0 # 0.0 means mixed-precision. >= 1.0 means actual scale up/down.
# q_protection_ratio = 0.01 # 0.01 means 1% of the weights are protected.
# q_smoothing_strength = 0.5

# investigation = Investigation(short_model_name=short_model_name, repo_dir=repo_dir, n_bits=n_bits, q_group_size=q_group_size, q_protect=q_protect, q_protection_scale=q_protection_scale, q_protection_ratio=q_protection_ratio, q_smoothing_strength=q_smoothing_strength)


Baseline fp16 already run. Results: 27.567453384399414
Baseline awq already run. Results: 27.744619369506836
Baseline smoothquant already run. Results: 27.623729705810547
Baseline smoothquant-g already run. Results: 27.545448303222656
Baseline w4a4 already run. Results: 5028.27880859375
Baseline smooth-w4a4 already run. Results: 404.6375427246094
Baseline w8a8 already run. Results: 27.830705642700195
Setup W4A4 G128 already run. Results=36.52133560180664, SmoothResults=32.48965072631836
Setup W4A4 G128 AWQ-Mixed-NoAct already run. Results=29.88942527770996, SmoothResults=30.239757537841797
Setup W4A4 G128 AWQ-Mixed-Act already run. Results=29.791271209716797, SmoothResults=30.041492462158203
fp16,27.567453384399414
awq,27.744619369506836
smoothquant,27.623729705810547
smoothquant-g,27.545448303222656
w4a4,5028.27880859375
smooth-w4a4,404.6375427246094
w8a8,27.830705642700195
W4A4 G128,36.52133560180664
Smooth W4A4 G128,32.48965072631836
W4A4 G128 AWQ-Mixed-NoAct,29.88942527770996
Smoot

In [ ]:
# AWQ Investigate
short_model_name = "opt-125m"
repo_dir = "llm-awq"
awq_zoo = "mit-han-lab/awq-model-zoo"
awq_pt_name = f"{short_model_name}-w4-g128.pt"
from huggingface_hub import hf_hub_download
awq_pt_filename = hf_hub_download(repo_id=awq_zoo, filename=awq_pt_name, repo_type="dataset")
awq_pt_filename

from awq.quantize.pre_quant import apply_awq
import torch
awq_results = torch.load(awq_pt_filename, map_location="cpu")



dict_keys(['scale', 'clip'])

In [4]:
from smoothquant.investigate import sweep, report_sweep, make_setups

setups = make_setups()
print(setups[0])
print(setups[1])


{'n_bits': 4, 'q_group_size': 0, 'q_protect': False, 'q_protection_scale': 0.0, 'q_protection_ratio': 0.03, 'q_smoothing_strength': 0.5}
{'n_bits': 4, 'q_group_size': 128, 'q_protect': False, 'q_protection_scale': 0.0, 'q_protection_ratio': 0.03, 'q_smoothing_strength': 0.5}


In [2]:
base_res = investigation.evaluate_base_model(perp=True)
print(f"Base Result: {base_res}")

evaluating...: 100%|██████████| 15/15 [00:11<00:00,  1.32it/s]


Base Result: 29.11669921875


In [3]:
quantized_res = investigation.evaluate_quantized_model(perp=True)
print(f"Quantized Result: {quantized_res}")

evaluating...: 100%|██████████| 15/15 [00:11<00:00,  1.28it/s]


Quantized Result: 32.10663986206055


In [4]:
smooth_quantized_res = investigation.evaluate_smooth_model(perp=True)
print(f"Smooth Quantized Result: {smooth_quantized_res}")

evaluating...: 100%|██████████| 15/15 [00:11<00:00,  1.29it/s]


Smooth Quantized Result: 30.784055709838867


In [1]:
import torch
import tqdm
from torch import nn
from transformers.models.opt.modeling_opt import (
    OPTAttention,
    OPTDecoderLayer,
    OPTForCausalLM,
)
from transformers import GPT2Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import WQAQLinear, quantize_opt

/Users/amlatyr/Code/efficient_ml_class_project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this notebook, we simulate the 8-bit dynamic per-tensor weight and activation quantization with FP16, i.e., fake quantization. We have implemented the real 8-bit quantization with INT8 CUTLASS GEMM kernels for both PyTorch and FasterTransformer. Please stay tuned for the release.

The following is an evaluator to see the performance of the model. We use a toy dataset (the first 1000 examples in the validation set of the Lambada dataset) to evaluate the model. You can replace it with your own dataset. The conclusion should be the same.

**In this demo, we have simplified the evaluation by using the first 1,000 samples from the LAMBADA dataset's validation set. We employ the "Last Token Prediction Accuracy" as our evaluation metric. This approximate evaluation is intended for demonstration purposes, providing simple but meaningful comparisons of relative performance between methods. For a more strict assessment, we recommend using the [lm-eval-harness](https://github.com/EleutherAI/lm-evaluation-harness) to obtain the "Last Word Prediction Accuracy" for the LAMBADA dataset, which is the reported metric in our paper.**

In [2]:
class PerplexityEvaluator:
    def __init__(self, dataset, tokenizer, device, n_samples=40):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        self.dataset = tokenizer(
            "\n\n".join(dataset["text"]), return_tensors="pt"
        ).input_ids.to(device)

        self.n_samples = n_samples

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        testenc = self.dataset
        nsamples = self.n_samples
        model = model.eval()

        nlls = []
        for i in tqdm.tqdm(range(nsamples), desc="evaluating..."):
            batch = testenc[:, (i * 2048):((i + 1) * 2048)].to(model.device)
            with torch.no_grad():
                lm_logits = model(batch).logits
            shift_logits = lm_logits[:, :-1, :].contiguous().float()
            shift_labels = testenc[:, (i * 2048):((i + 1) * 2048)][:, 1:]
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            neg_log_likelihood = loss.float() * 2048
            nlls.append(neg_log_likelihood)

        return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))

class AccuracyEvaluator:
    def __init__(self, dataset, tokenizer, device):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        # tokenize the dataset
        def tokenize_function(examples):
            example = self.tokenizer(examples["text"])
            return example

        self.dataset = self.dataset.map(tokenize_function, batched=True)
        self.dataset.set_format(type="torch", columns=["input_ids"])

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        # The task is to predict the last word of the input.
        total, hit = 0, 0
        for batch in self.dataset:
            input_ids = batch["input_ids"].to(self.device).unsqueeze(0)
            label = input_ids[:, -1]
            outputs = model(input_ids)
            last_token_logits = outputs.logits[:, -2, :]
            pred = last_token_logits.argmax(dim=-1)
            total += label.size(0)
            hit += (pred == label).sum().item()
        acc = hit / total
        return acc

In [3]:
from datasets import load_dataset

model_name = "facebook/opt-125m"
acc_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
perp_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
acc_dataset = load_dataset("lambada", split="validation[:40]")
perp_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
acc_evaluator = AccuracyEvaluator(acc_dataset, acc_tokenizer, device)
perp_evaluator = PerplexityEvaluator(perp_dataset, perp_tokenizer, device, n_samples=15)

## FP16 Model Accuracy

Let's first check the performance of the original FP16 model.

In [4]:
model_fp16 = OPTForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)
res_fp16 = perp_evaluator.evaluate(model_fp16)
print(f"Original model (fp16) result: {res_fp16}")

evaluating...: 100%|██████████| 15/15 [00:11<00:00,  1.30it/s]


Original model (fp16) result: 29.11669921875


We then quantize the model to W8A8 and check the performance.

## Naive W8A8 Quantized Model Accuracy

In [13]:
model_fp16 = OPTForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)

n_bits = 4
q_group_size = 128 # 0 means no group
q_protect = True # False means no protection
q_protection_ratio = 0.01 # 0.01 means 1% of the weights are protected.
q_protection_scale = 0.0 # 0.0 mixed-precision. >1.0 means scale up/down.
q_name = f"W{n_bits}A{n_bits}"
q_model = quantize_opt(
    model_fp16,
    n_bits=n_bits,
    q_group_size=q_group_size,
    q_protect=q_protect,
    q_protection_ratio=q_protection_ratio,
    q_protection_scale=q_protection_scale,
)
q_res = perp_evaluator.evaluate(q_model)
print(f"Naive {q_name} quantized model result: {q_res}")

evaluating...: 100%|██████████| 15/15 [00:23<00:00,  1.58s/it]

Naive W4A4 quantized model result: 42.792396545410156


In [14]:
print(q_model)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): WALinear(768, 768, bias=True, weight_quant=protected_group_quant_128, act_quant=group_quant_128, output_quant=group_quant_128)
            (v_proj): WALinear(768, 768, bias=True, weight_quant=protected_group_quant_128, act_quant=group_quant_128, output_quant=group_quant_128)
            (q_proj): WALinear(768, 768, bias=True, weight_quant=protected_group_quant_128, act_quant=group_quant_128, output_quant=group_quant_128)
            (out_proj): WALinear(768, 768, bias=True, weight_quant=protected_group_quant_128, act_quant=group_quant_128, output_quant=None)
          )
          (activation_fn)

We can see there is a significant accuracy drop. This is consistent with LLM.int8()'s finding: when the model size increases larger than 6.7B, systematic outliers will emerge in activations, which makes fully INT8 quantization impossible.

## SmoothQuant W8A8 Quantized Model Accuracy

Let's smooth the model, quantize it, and check the performance! In `../act_scales`, we provide the activation scales for OPT and BLOOM models. You can also use this notebook to test quantizing those models.

In [11]:
model = OPTForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)
scales_path = "smoothquant/act_scales/opt-125m.pt"
act_scales = torch.load(scales_path)
smooth_lm(model, act_scales, 0.5)
q_model_smooth = quantize_opt(
    model,
    n_bits=n_bits,
    q_group_size=q_group_size,
    q_protect=q_protect,
    q_protection_ratio=q_protection_ratio,
    q_protection_scale=q_protection_scale,
)
q_res_smooth = perp_evaluator.evaluate(q_model_smooth)
print(f"Smoothed {q_name} quantized model result: {q_res_smooth}")

evaluating...: 100%|██████████| 15/15 [00:25<00:00,  1.70s/it]

Smoothed W4A4 quantized model result: 36.66819763183594


We can see the smoothed model has the same accuracy as the FP16 model. This is because SmoothQuant smooths the outliers in activations and moves the quantization difficulty from activations to weights.

SmoothQuant W8A8 quantized model accuracy: 0.793
